# Introducción a MPI

Bibliografía
==

<a href="http://computing.llnl.gov/tutorials/mpi/">Message Passing Interface (MPI)</a>

<a href="http://mpitutorial.com/tutorials/running-an-mpi-cluster-within-a-lan/">Running an MPI Cluster within a LAN</a>

<a href="http://condor.cc.ku.edu/~grobe/docs/intro-MPI-C.shtml">An introduction to the 
Message Passing Interface (MPI) using C</a>



Hello world
==

In [5]:
%%writefile hello.c

#include <stdio.h>
#include <mpi.h>

int main(int argc, char **argv) 
{
  int ierr;

  ierr = MPI_Init(&argc, &argv);
  printf("Hello world\n"); 

  ierr = MPI_Finalize();
    return 0;
}

Overwriting hello.c


In [28]:
%%bash
mpicc hello.c -o hello

In [30]:
%%bash
mpirun -np 4 hello

Hello world
Hello world
Hello world
Hello world


In [39]:
%%writefile hello2.c
#include <stdio.h>
#include <mpi.h>

int main(int argc, char **argv)
{
  int ierr, num_procs, my_id;

  ierr = MPI_Init(&argc, &argv);

  /* find out MY process ID, and how many processes were started. */

  ierr = MPI_Comm_rank(MPI_COMM_WORLD, &my_id);
  ierr = MPI_Comm_size(MPI_COMM_WORLD, &num_procs);

  printf("Hello world! I'm process %i out of %i processes\n", 
     my_id, num_procs);

  ierr = MPI_Finalize();
  
  return 0;
}

Overwriting hello2.c


In [40]:
%%bash
mpicc hello2.c -o hello2

In [41]:
%%bash
mpirun -np 4 hello2

Hello world! I'm process 1 out of 4 processes
Hello world! I'm process 3 out of 4 processes
Hello world! I'm process 2 out of 4 processes
Hello world! I'm process 0 out of 4 processes


# Código en función del Rank

In [43]:
%%writefile different_task.c   
    #include <mpi.h>

int main(int argc, char **argv)
   {
      int my_id, root_process, ierr, num_procs;
      MPI_Status status;

      /* Create child processes, each of which has its own variables.
       * From this point on, every process executes a separate copy
       * of this program.  Each process has a different process ID,
       * ranging from 0 to num_procs minus 1, and COPIES of all
       * variables defined in the program. No variables are shared.
       **/

      ierr = MPI_Init(&argc, &argv);
     
      /* find out MY process ID, and how many processes were started. */
      
      ierr = MPI_Comm_rank(MPI_COMM_WORLD, &my_id);
      ierr = MPI_Comm_size(MPI_COMM_WORLD, &num_procs);

      if( my_id == 0 ) {

         /* do some work as process 0 */
      }
      else if( my_id == 1 ) {

         /* do some work as process 1 */
      }
      else if( my_id == 2 ) {

         /* do some work as process 2 */ 
      } 
      else {

         /* do this work in any remaining processes */
      }
      /* Stop this process */

      ierr = MPI_Finalize();
        
      return 0;
   }

Writing different_task.c


In [45]:
%%bash
mpicc different_task.c -o different_task
mpirun -np 4 different_task

# Démosle una oportunidad a python

"*La optimización prematura es la raíz de todos los males*"  Donald Knuth

Una ventaja de C es la velocidad de ejecución de las aplicaciones, una desventaja es que la ingeniería de software es demasiado costosa como para desarrollar aplicaciones complejas. Lo ideal es hacer un prototipo rápido en un lenguaje de ingeniería de software más ágil y luego de tener resultados favorables hacer una implementación que se oriente a la velocidad en los resultados.

En esta parte veremos algunos ejemplos de similares resultados a los anteriores pero en **python 2.7**

## Recursos

https://bitbucket.org/mpi4py/mpi4py

http://pythonhosted.org/mpi4py/mpi4py.pdf

http://pythonhosted.org/mpi4py/usrman/tutorial.html


In [56]:
%%writefile ej0.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    
    data = {'a': 7, 'b': 3.14}

    comm.send(data, dest=1, tag=11)

elif rank == 1:
    
    data = comm.recv(source=0, tag=11)
    print data


Overwriting ej0.py


In [57]:
%%bash
mpirun -np 2 python ej0.py

{'a': 7, 'b': 3.14}


In [66]:
%%writefile ej_non_blocking.py

#parecido al patron de diseno promesa
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'a': 7, 'b': 3.14}
    req = comm.isend(data, dest=1, tag=11)
    req.wait()
elif rank == 1:
    req = comm.irecv(source=0, tag=11)
    data = req.wait()
    print data

Overwriting ej_non_blocking.py


In [67]:
%%bash
mpirun -np 2 python ej_non_blocking.py

{'a': 7, 'b': 3.14}


In [72]:
%%writefile ej_numpy.py
from mpi4py import MPI
import numpy

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# passing MPI datatypes explicitly
if rank == 0:
    data = numpy.arange(1000, dtype='i')
    comm.Send([data, MPI.INT], dest=1, tag=77)
elif rank == 1:
    data = numpy.empty(1000, dtype='i')
    comm.Recv([data, MPI.INT], source=0, tag=77)

# automatic MPI datatype discovery
if rank == 0:
    data = numpy.arange(100, dtype=numpy.float64)
    comm.Send(data, dest=1, tag=13)
elif rank == 1:
    data = numpy.empty(100, dtype=numpy.float64)
    comm.Recv(data, source=0, tag=13)
    print data

Overwriting ej_numpy.py


In [73]:
%%bash
mpirun -n 2 python ej_numpy.py

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.  14.
  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.  28.  29.
  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.  42.  43.  44.
  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.  56.  57.  58.  59.
  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.  70.  71.  72.  73.  74.
  75.  76.  77.  78.  79.  80.  81.  82.  83.  84.  85.  86.  87.  88.  89.
  90.  91.  92.  93.  94.  95.  96.  97.  98.  99.]


# Collective comunication

Usar send y recv es una forma primitiva de resolver problemas, para aplicar el verdadero map-reduce debemos saber dividir la data en función de la cantidad de procesadores y enviarlos a cada uno de los nodos esclavos.

In [74]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = {'key1' : [7, 2.72, 2+3j],
            'key2' : ( 'abc', 'xyz')}
else:
    data = None
data = comm.bcast(data, root=0)

In [75]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

if rank == 0:
    data = [(i+1)**2 for i in range(size)]
else:
    data = None
data = comm.scatter(data, root=0)
assert data == (rank+1)**2

In [76]:
from mpi4py import MPI

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

data = (rank+1)**2
data = comm.gather(data, root=0)
if rank == 0:
    for i in range(size):
        assert data[i] == (i+1)**2
else:
    assert data is None

In [77]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

if rank == 0:
    data = np.arange(100, dtype='i')
else:
    data = np.empty(100, dtype='i')
comm.Bcast(data, root=0)
for i in range(100):
    assert data[i] == i

In [78]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = None
if rank == 0:
    sendbuf = np.empty([size, 100], dtype='i')
    sendbuf.T[:,:] = range(size)
recvbuf = np.empty(100, dtype='i')
comm.Scatter(sendbuf, recvbuf, root=0)
assert np.allclose(recvbuf, rank)

In [79]:
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
size = comm.Get_size()
rank = comm.Get_rank()

sendbuf = np.zeros(100, dtype='i') + rank
recvbuf = None
if rank == 0:
    recvbuf = np.empty([size, 100], dtype='i')
comm.Gather(sendbuf, recvbuf, root=0)
if rank == 0:
    for i in range(size):
        assert np.allclose(recvbuf[i,:], i)

In [80]:
from mpi4py import MPI
import numpy

def matvec(comm, A, x):
    m = A.shape[0] # local rows
    p = comm.Get_size()
    xg = numpy.zeros(m*p, dtype='d')
    comm.Allgather([x,  MPI.DOUBLE],
                   [xg, MPI.DOUBLE])
    y = numpy.dot(A, xg)
    return y